In [1]:
%load_ext autoreload
%autoreload 2

## Open questions

* How to show all the resulting contexts for all examples to the user?
* How to show the intermediates to the user?
* How to allow the user to compare two prompt versions?
* How to allow the user to provide feedback on whether something is a good result or not?
* How to manage the version blowup when automatically saving each .run() result as a version?

Optimizations:
* Stream intermediate outputs to the user as they are generated, allowing users to cancel early if they see an error
* Track control flow to identify opportunities for reducing prompt lengths/costs

In [2]:
%load_ext gears.extension 
%reload_ext gears.extension

In [3]:
from gears import Example, Session, Gear, History
from gears.llms import AzureOpenAIChat
from gears.utils import extract_first_json
from dotenv import load_dotenv
import openai
import os

# Set Azure keys
load_dotenv()
openai.api_type = "azure"
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_key = os.getenv("OPENAI_AZURE_API_KEY")
openai.api_version = "2023-07-01-preview"

In [4]:
class Context(Example):
    city: str

In [5]:
session = Session()

session.add_example(Context(city="Paris"))

In [6]:
session.examples

[Context(id='ec19cbd0-6457-4f21-89a1-f0beedc0d389', city='Paris')]

In [7]:
%%gear --root
class Ideation(Gear):
    def __init__(self):
        super().__init__(AzureOpenAIChat(deployment_id="gpt-35-turbo"))
    
    def prompt(self, context: Context):
        return "Suggest some types of interactions that one might encounter when visiting the city {{ context.city }} as a tourist. Return a JSON with key `interactions` and value equal to a list of strings."
    
    def transform(self, reply: dict, context: Context):
        interactions = extract_first_json(reply["choices"][0]["message"]["content"].strip())["interactions"]
        context.interactions = interactions
        
        return context
    
    def switch(self, context: Context):
        return EnglishQuestion()

In [8]:
%%gear
class EnglishQuestion(Gear):
    def __init__(self):
        super().__init__(AzureOpenAIChat(deployment_id="gpt-35-turbo"))
        
    def prompt(self, context: Context):
        return "For each interaction, write a question that a tourist might ask about the interaction. Return a JSON with key `questions` and value equal to a list of strings."
    
    def transform(self, reply: dict, context: Context):
        questions = extract_first_json(reply["choices"][0]["message"]["content"].strip())["questions"]
        context.questions = questions
        
        return context

    def switch(self, context: Context):
        return LanguageQuestion()

In [10]:
%%gear
class LanguageQuestion(Gear):
    def __init__(self):
        super().__init__(AzureOpenAIChat(deployment_id="gpt-35-turbo"))
    
    def prompt(self, context: Context):
        return "Consider the most popular language in the city {{ context.city }}. Then translate each question into this language. If the most popular language of the city is English, then return the same English questions. Return a JSON with key `translations` and value equal to a list of strings."
    
    def transform(self, reply: dict, context: Context):
        translated_questions = extract_first_json(reply["choices"][0]["message"]["content"].strip())["translations"]
        context.translated_questions = translated_questions
        
        return context

In [11]:
# Run session

await session.run(version=True)

[19:17:21] INFO     Running LanguageQuestion on example ec19cbd0-6457-4f21-89a1-f0beedc0d389.        ]8;id=265175;file:///Users/shreyashankar/Documents/projects/gears/gears/session.py\session.py]8;;\:]8;id=684454;file:///Users/shreyashankar/Documents/projects/gears/gears/session.py#151\151]8;;\

[19:17:23] INFO     message='OpenAI API response'                                                        ]8;id=890003;file:///Users/shreyashankar/miniforge3/envs/gearstest/lib/python3.9/site-packages/openai/util.py\util.py]8;;\:]8;id=115753;file:///Users/shreyashankar/miniforge3/envs/gearstest/lib/python3.9/site-packages/openai/util.py#67\67]8;;\
                    path=https://shreyashankar.openai.azure.com//openai/deployments/gpt-35-turbo/chat/co           
                    mpletions?api-version=2023-07-01-preview processing_ms=1825.4483                               
                    request_id=fe86cb53-e4c3-4df4-807b-f172899135d3 response_code=200                              

           INFO     This run cost $0.0002. The total cost of the session so far is $0.0011.          ]8;id=396968;file:///Users/shreyashankar/Documents/projects/gears/gears/session.py\session.py]8;;\:]8;id=562673;file:///Users/shreyashankar/Documents/projects/gears/gears/session.py#224\224]8;;\

           INFO     Saving workflow 1 to session.                                                    ]8;id=523562;file:///Users/shreyashankar/Documents/projects/gears/gears/session.py\session.py]8;;\:]8;id=485888;file:///Users/shreyashankar/Documents/projects/gears/gears/session.py#245\245]8;;\

[Context(id='ec19cbd0-6457-4f21-89a1-f0beedc0d389', city='Paris', interactions=['Exploring cultural and historical landmarks', 'Trying local cuisine at restaurants and cafes', 'Shopping at local markets and boutiques', 'Attending music and art festivals', 'Visiting museums and art galleries', 'Taking guided tours of the city', 'Engaging in outdoor activities such as hiking or biking', 'Attending sporting events', 'Meeting local residents and learning about their customs and traditions'], questions=['What are some cultural and historical landmarks that I should visit in {{ context.city }}?', 'What are some local dishes that I have to try while in {{ context.city }}?', 'Are there any interesting markets or boutiques to explore in {{ context.city }}?', 'What music and art festivals are taking place during my visit to {{ context.city }}?', 'What museums and art galleries should I visit in {{ context.city }}?', 'What kind of guided tours are available in {{ context.city }}?', 'What outdoor 

In [12]:
# Run session again (should not ping OpenAI API)

await session.run()

[19:17:32] INFO     No new gears ran. The total cost of the session so far is $0.0011.               ]8;id=394315;file:///Users/shreyashankar/Documents/projects/gears/gears/session.py\session.py]8;;\:]8;id=322867;file:///Users/shreyashankar/Documents/projects/gears/gears/session.py#228\228]8;;\

[Context(id='ec19cbd0-6457-4f21-89a1-f0beedc0d389', city='Paris', interactions=['Exploring cultural and historical landmarks', 'Trying local cuisine at restaurants and cafes', 'Shopping at local markets and boutiques', 'Attending music and art festivals', 'Visiting museums and art galleries', 'Taking guided tours of the city', 'Engaging in outdoor activities such as hiking or biking', 'Attending sporting events', 'Meeting local residents and learning about their customs and traditions'], questions=['What are some cultural and historical landmarks that I should visit in {{ context.city }}?', 'What are some local dishes that I have to try while in {{ context.city }}?', 'Are there any interesting markets or boutiques to explore in {{ context.city }}?', 'What music and art festivals are taking place during my visit to {{ context.city }}?', 'What museums and art galleries should I visit in {{ context.city }}?', 'What kind of guided tours are available in {{ context.city }}?', 'What outdoor 

In [13]:
# Add an example

session.add_example(Context(city="London"))

In [14]:
# Run session again (should ping OpenAI API only once, for the new example)

await session.run(version=True)

[19:17:36] INFO     Running Ideation on example ec19cbd0-6457-4f21-89a1-f0beedc0d389.                ]8;id=501509;file:///Users/shreyashankar/Documents/projects/gears/gears/session.py\session.py]8;;\:]8;id=329306;file:///Users/shreyashankar/Documents/projects/gears/gears/session.py#151\151]8;;\

[19:17:39] INFO     message='OpenAI API response'                                                        ]8;id=296534;file:///Users/shreyashankar/miniforge3/envs/gearstest/lib/python3.9/site-packages/openai/util.py\util.py]8;;\:]8;id=719619;file:///Users/shreyashankar/miniforge3/envs/gearstest/lib/python3.9/site-packages/openai/util.py#67\67]8;;\
                    path=https://shreyashankar.openai.azure.com//openai/deployments/gpt-35-turbo/chat/co           
                    mpletions?api-version=2023-07-01-preview processing_ms=2298.8082                               
                    request_id=bc4d3649-1388-4c59-a7f4-5f79c5f57f0a response_code=200                              

           INFO     Running EnglishQuestion on example ec19cbd0-6457-4f21-89a1-f0beedc0d389.         ]8;id=457682;file:///Users/shreyashankar/Documents/projects/gears/gears/session.py\session.py]8;;\:]8;id=276411;file:///Users/shreyashankar/Documents/projects/gears/gears/session.py#151\151]8;;\

[19:17:45] INFO     message='OpenAI API response'                                                        ]8;id=926952;file:///Users/shreyashankar/miniforge3/envs/gearstest/lib/python3.9/site-packages/openai/util.py\util.py]8;;\:]8;id=472428;file:///Users/shreyashankar/miniforge3/envs/gearstest/lib/python3.9/site-packages/openai/util.py#67\67]8;;\
                    path=https://shreyashankar.openai.azure.com//openai/deployments/gpt-35-turbo/chat/co           
                    mpletions?api-version=2023-07-01-preview processing_ms=5333.0429                               
                    request_id=3cf251c0-2599-4738-9259-0e3627a86a41 response_code=200                              

           INFO     Running LanguageQuestion on example ec19cbd0-6457-4f21-89a1-f0beedc0d389.        ]8;id=920661;file:///Users/shreyashankar/Documents/projects/gears/gears/session.py\session.py]8;;\:]8;id=401971;file:///Users/shreyashankar/Documents/projects/gears/gears/session.py#151\151]8;;\

[19:17:48] INFO     message='OpenAI API response'                                                        ]8;id=190683;file:///Users/shreyashankar/miniforge3/envs/gearstest/lib/python3.9/site-packages/openai/util.py\util.py]8;;\:]8;id=750533;file:///Users/shreyashankar/miniforge3/envs/gearstest/lib/python3.9/site-packages/openai/util.py#67\67]8;;\
                    path=https://shreyashankar.openai.azure.com//openai/deployments/gpt-35-turbo/chat/co           
                    mpletions?api-version=2023-07-01-preview processing_ms=2687.7653                               
                    request_id=264799d4-5e4d-4519-82ec-1eb4dde20461 response_code=200                              

RuntimeError: coroutine raised StopIteration

In [ ]:
session.cost

0.004371

In [12]:
session.versions[0]

{'root': 'Ideation',
 'code': {'Ideation': 'class Ideation(Gear):\n    def __init__(self):\n        super().__init__(AzureOpenAIChat(deployment_id="gpt-35-turbo"))\n    \n    def template(self, context: Context):\n        return "Suggest some types of interactions that one might encounter when visiting the city {{ context.city }} as a tourist. Return a JSON with key `interactions` and value equal to a list of strings."\n    \n    def transform(self, reply: dict, context: Context):\n        interactions = extract_first_json(reply["choices"][0]["message"]["content"].strip())["interactions"]\n        context.interactions = interactions\n        \n        return context\n    \n    def switch(self, context: Context):\n        return EnglishQuestion()\n',
  'EnglishQuestion': 'class EnglishQuestion(Gear):\n    def __init__(self):\n        super().__init__(AzureOpenAIChat(deployment_id="gpt-35-turbo"))\n    \n    def template(self, context: Context):\n        return "For each interaction, wri

In [13]:
session.add_example(Context(city="Berlin"))

In [18]:
import pandas as pd

res = await session.run()
display(pd.DataFrame(res))

[14:25:31] INFO     No new gears ran. The total cost of the session so far is $0.0077.               ]8;id=783154;file:///Users/shreyashankar/Documents/projects/gears/gears/session.py\session.py]8;;\:]8;id=612702;file:///Users/shreyashankar/Documents/projects/gears/gears/session.py#195\195]8;;\

,0,1,2,3,4
0,"(id, d3ee24cc-d899-433e-96f0-4663d8fee658)","(city, Paris)","(interactions, [Ordering food at a restaurant,...","(questions, [What are some traditional French ...","(translated_questions, [Quels plats traditionn..."
1,"(id, d3ee24cc-d899-433e-96f0-4663d8fee658)","(city, Berlin)","(interactions, [Tour of the Berlin Wall, Visit...","(questions, [What is the history of the Berlin...","(translated_questions, [Was ist die Geschichte..."
2,"(id, d3ee24cc-d899-433e-96f0-4663d8fee658)","(city, London)","(interactions, [Touring famous landmarks such ...","(questions, [What is the history behind the la...","(translated_questions, [What is the history be..."


In [20]:
pd.DataFrame([r.dict() for r in res])

/var/folders/nq/ldkhrrws0xb9whw7b6rpzhc00000gn/T/ipykernel_23755/113781542.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.0.3/migration/
  pd.DataFrame([r.dict() for r in res])


,id,city,interactions,questions,translated_questions
0,d3ee24cc-d899-433e-96f0-4663d8fee658,Paris,"[Ordering food at a restaurant, Asking for dir...",[What are some traditional French dishes that ...,[Quels plats traditionnels français devrais-je...
1,d3ee24cc-d899-433e-96f0-4663d8fee658,Berlin,"[Tour of the Berlin Wall, Visit to Brandenburg...","[What is the history of the Berlin Wall?, What...","[Was ist die Geschichte der Berliner Mauer?, W..."
2,d3ee24cc-d899-433e-96f0-4663d8fee658,London,"[Touring famous landmarks such as Big Ben, the...","[What is the history behind the landmark?, Wha...","[What is the history behind the landmark?, Wha..."
